## Imports

In [1]:
# pip install --upgrade scikit-learn

In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.3
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [5]:
df      = pd.read_csv( DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv( DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [7]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [8]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [9]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda x: x.split(',')[1].strip().split('.')[0]

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)
print(df['Title'].unique())
df_test['Title'].unique()

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']


array(['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Col', 'Rev', 'Dr', 'Dona'],
      dtype=object)

In [10]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [11]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Master",
    "Don": "Mr",
    "Sir" : "Mr",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Mrs",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Mrs",
    "Dona" : "Mrs"
}

In [12]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df['Title'].values[886]
df['Title'] = df['Title'].map(lambda i : title_dictionary[i])
df['Title'].values[886]
df_test['Title'] = df_test['Title'].map(lambda x : title_dictionary[x])
df['Title']

PassengerId
1           Mr
2          Mrs
3         Miss
4          Mrs
5           Mr
        ...   
887    Officer
888       Miss
889       Miss
890         Mr
891         Mr
Name: Title, Length: 891, dtype: object

In [13]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [14]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [15]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [16]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [17]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [18]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE


num_4_treeModels = pipeline.Pipeline([
    ('imputer',impute.SimpleImputer(strategy='mean'))
])
print(type(num_4_treeModels))

cat_4_treeModels = pipeline.Pipeline([('imputer',impute.SimpleImputer(strategy='constant',fill_value='missing')),
                          ('enc', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels , num_vars),
    ('cat', cat_4_treeModels , cat_vars)], 
    remainder='drop')

<class 'sklearn.pipeline.Pipeline'>


In [19]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [20]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install catboost

In [23]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [24]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":
  "Random Forest":
  "AdaBoost":
  "Skl GBM":
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
""";


# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(max_depth=6),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM":  LGBMClassifier(),
  "CatBoost": CatBoostClassifier()}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('enc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [25]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [26]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=0)
#print(x_train.head())
#print(y_train.head())
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    #print(model)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    #model.transform(x_val)
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\Dilan\anaconda3\envs\Strive_School_AI_March_2021\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:15:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6875442	total: 177ms	remaining: 2m 57s
1:	learn: 0.6821340	total: 188ms	remaining: 1m 33s
2:	learn: 0.6775612	total: 190ms	remaining: 1m 3s
3:	learn: 0.6718929	total: 194ms	remaining: 48.3s
4:	learn: 0.6669486	total: 197ms	remaining: 39.2s
5:	learn: 0.6619068	total: 199ms	remaining: 33s
6:	learn: 0.6564104	total: 203ms	remaining: 28.8s
7:	learn: 0.6513312	total: 206ms	remaining: 25.6s
8:	learn: 0.6467002	total: 210ms	remaining: 23.1s
9:	learn: 0.6416217	total: 213ms	remaining: 21.1s
10:	learn: 0.6363435	total: 216ms	remaining: 19.4s
11:	learn: 0.6318981	total: 219ms	remaining: 18s
12:	learn: 0.6273955	total: 222ms	remaining: 16.8s
13:	learn

185:	learn: 0.3935339	total: 696ms	remaining: 3.05s
186:	learn: 0.3931823	total: 699ms	remaining: 3.04s
187:	learn: 0.3926861	total: 703ms	remaining: 3.03s
188:	learn: 0.3923628	total: 706ms	remaining: 3.03s
189:	learn: 0.3918204	total: 709ms	remaining: 3.02s
190:	learn: 0.3914300	total: 712ms	remaining: 3.02s
191:	learn: 0.3911275	total: 715ms	remaining: 3.01s
192:	learn: 0.3907912	total: 718ms	remaining: 3s
193:	learn: 0.3905715	total: 721ms	remaining: 3s
194:	learn: 0.3901585	total: 724ms	remaining: 2.99s
195:	learn: 0.3896518	total: 726ms	remaining: 2.98s
196:	learn: 0.3893453	total: 730ms	remaining: 2.97s
197:	learn: 0.3890080	total: 733ms	remaining: 2.97s
198:	learn: 0.3887323	total: 735ms	remaining: 2.96s
199:	learn: 0.3884030	total: 738ms	remaining: 2.95s
200:	learn: 0.3880795	total: 740ms	remaining: 2.94s
201:	learn: 0.3878200	total: 743ms	remaining: 2.94s
202:	learn: 0.3876151	total: 746ms	remaining: 2.93s
203:	learn: 0.3873586	total: 749ms	remaining: 2.92s
204:	learn: 0.3870

366:	learn: 0.3550293	total: 1.2s	remaining: 2.08s
367:	learn: 0.3549215	total: 1.21s	remaining: 2.07s
368:	learn: 0.3547427	total: 1.21s	remaining: 2.07s
369:	learn: 0.3546107	total: 1.21s	remaining: 2.06s
370:	learn: 0.3544855	total: 1.22s	remaining: 2.06s
371:	learn: 0.3542883	total: 1.22s	remaining: 2.06s
372:	learn: 0.3541848	total: 1.22s	remaining: 2.05s
373:	learn: 0.3540561	total: 1.22s	remaining: 2.05s
374:	learn: 0.3538709	total: 1.23s	remaining: 2.05s
375:	learn: 0.3538075	total: 1.23s	remaining: 2.04s
376:	learn: 0.3535131	total: 1.23s	remaining: 2.04s
377:	learn: 0.3533470	total: 1.24s	remaining: 2.03s
378:	learn: 0.3532188	total: 1.24s	remaining: 2.03s
379:	learn: 0.3531951	total: 1.24s	remaining: 2.02s
380:	learn: 0.3530957	total: 1.24s	remaining: 2.02s
381:	learn: 0.3529107	total: 1.25s	remaining: 2.02s
382:	learn: 0.3527679	total: 1.25s	remaining: 2.01s
383:	learn: 0.3526622	total: 1.25s	remaining: 2.01s
384:	learn: 0.3525453	total: 1.25s	remaining: 2s
385:	learn: 0.35

557:	learn: 0.3306302	total: 1.72s	remaining: 1.36s
558:	learn: 0.3303717	total: 1.72s	remaining: 1.36s
559:	learn: 0.3302354	total: 1.72s	remaining: 1.35s
560:	learn: 0.3301008	total: 1.73s	remaining: 1.35s
561:	learn: 0.3299797	total: 1.73s	remaining: 1.35s
562:	learn: 0.3299047	total: 1.73s	remaining: 1.34s
563:	learn: 0.3297887	total: 1.74s	remaining: 1.34s
564:	learn: 0.3296730	total: 1.74s	remaining: 1.34s
565:	learn: 0.3295675	total: 1.74s	remaining: 1.33s
566:	learn: 0.3292600	total: 1.75s	remaining: 1.33s
567:	learn: 0.3290978	total: 1.75s	remaining: 1.33s
568:	learn: 0.3290658	total: 1.75s	remaining: 1.33s
569:	learn: 0.3288739	total: 1.75s	remaining: 1.32s
570:	learn: 0.3287541	total: 1.76s	remaining: 1.32s
571:	learn: 0.3286257	total: 1.76s	remaining: 1.32s
572:	learn: 0.3283502	total: 1.76s	remaining: 1.31s
573:	learn: 0.3282614	total: 1.76s	remaining: 1.31s
574:	learn: 0.3280409	total: 1.77s	remaining: 1.31s
575:	learn: 0.3279379	total: 1.77s	remaining: 1.3s
576:	learn: 0

741:	learn: 0.3084905	total: 2.23s	remaining: 774ms
742:	learn: 0.3084368	total: 2.23s	remaining: 771ms
743:	learn: 0.3083300	total: 2.23s	remaining: 768ms
744:	learn: 0.3082730	total: 2.23s	remaining: 765ms
745:	learn: 0.3081707	total: 2.24s	remaining: 762ms
746:	learn: 0.3081043	total: 2.24s	remaining: 759ms
747:	learn: 0.3080700	total: 2.25s	remaining: 757ms
748:	learn: 0.3078790	total: 2.25s	remaining: 754ms
749:	learn: 0.3077535	total: 2.25s	remaining: 751ms
750:	learn: 0.3075784	total: 2.25s	remaining: 748ms
751:	learn: 0.3075209	total: 2.26s	remaining: 744ms
752:	learn: 0.3073673	total: 2.26s	remaining: 741ms
753:	learn: 0.3072789	total: 2.26s	remaining: 738ms
754:	learn: 0.3070902	total: 2.27s	remaining: 735ms
755:	learn: 0.3069533	total: 2.27s	remaining: 732ms
756:	learn: 0.3068744	total: 2.27s	remaining: 729ms
757:	learn: 0.3067994	total: 2.27s	remaining: 726ms
758:	learn: 0.3066569	total: 2.28s	remaining: 723ms
759:	learn: 0.3065480	total: 2.28s	remaining: 720ms
760:	learn: 

924:	learn: 0.2879735	total: 2.74s	remaining: 222ms
925:	learn: 0.2879196	total: 2.74s	remaining: 219ms
926:	learn: 0.2877286	total: 2.74s	remaining: 216ms
927:	learn: 0.2875768	total: 2.75s	remaining: 213ms
928:	learn: 0.2875165	total: 2.75s	remaining: 210ms
929:	learn: 0.2874034	total: 2.75s	remaining: 207ms
930:	learn: 0.2872871	total: 2.76s	remaining: 204ms
931:	learn: 0.2871842	total: 2.76s	remaining: 201ms
932:	learn: 0.2870510	total: 2.76s	remaining: 198ms
933:	learn: 0.2869615	total: 2.77s	remaining: 195ms
934:	learn: 0.2866832	total: 2.77s	remaining: 192ms
935:	learn: 0.2865423	total: 2.77s	remaining: 189ms
936:	learn: 0.2864316	total: 2.77s	remaining: 186ms
937:	learn: 0.2862661	total: 2.78s	remaining: 184ms
938:	learn: 0.2862455	total: 2.78s	remaining: 181ms
939:	learn: 0.2861992	total: 2.78s	remaining: 178ms
940:	learn: 0.2861437	total: 2.79s	remaining: 175ms
941:	learn: 0.2860451	total: 2.79s	remaining: 172ms
942:	learn: 0.2858789	total: 2.79s	remaining: 169ms
943:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,84.229249,3.403901
2,Skl GBM,84.916201,82.325428,0.193881
3,Random Forest,83.240223,80.961792,0.300813
4,Skl HistGBM,83.240223,81.231884,1.156285
5,LightGBM,82.681564,80.777339,0.165898
6,Extra Trees,82.122905,80.322793,0.269834
7,XGBoost,82.122905,80.322793,0.261840
8,AdaBoost,81.564246,80.678524,0.163899
9,Decision Tree,79.888268,77.694335,0.046968


In [27]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [29]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""
print(x.shape)
print(y.shape)

for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
     
    pred = model_selection.cross_val_predict(model, x, y,cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

(891, 8)
(891,)


C:\Users\Dilan\anaconda3\envs\Strive_School_AI_March_2021\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:20:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dilan\anaconda3\envs\Strive_School_AI_March_2021\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:20:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

327:	learn: 0.3481490	total: 1s	remaining: 2.06s
328:	learn: 0.3478647	total: 1.01s	remaining: 2.06s
329:	learn: 0.3477306	total: 1.01s	remaining: 2.05s
330:	learn: 0.3476788	total: 1.01s	remaining: 2.05s
331:	learn: 0.3475548	total: 1.02s	remaining: 2.04s
332:	learn: 0.3474164	total: 1.02s	remaining: 2.04s
333:	learn: 0.3474018	total: 1.02s	remaining: 2.04s
334:	learn: 0.3472464	total: 1.02s	remaining: 2.03s
335:	learn: 0.3471442	total: 1.03s	remaining: 2.03s
336:	learn: 0.3469770	total: 1.03s	remaining: 2.03s
337:	learn: 0.3468387	total: 1.03s	remaining: 2.02s
338:	learn: 0.3467815	total: 1.04s	remaining: 2.02s
339:	learn: 0.3466943	total: 1.04s	remaining: 2.02s
340:	learn: 0.3465408	total: 1.04s	remaining: 2.01s
341:	learn: 0.3464677	total: 1.04s	remaining: 2.01s
342:	learn: 0.3462422	total: 1.05s	remaining: 2.01s
343:	learn: 0.3462325	total: 1.05s	remaining: 2s
344:	learn: 0.3460865	total: 1.05s	remaining: 2s
345:	learn: 0.3459339	total: 1.05s	remaining: 1.99s
346:	learn: 0.3457489

514:	learn: 0.3243164	total: 1.68s	remaining: 1.59s
515:	learn: 0.3240177	total: 1.69s	remaining: 1.58s
516:	learn: 0.3239239	total: 1.69s	remaining: 1.58s
517:	learn: 0.3238130	total: 1.7s	remaining: 1.58s
518:	learn: 0.3237662	total: 1.7s	remaining: 1.57s
519:	learn: 0.3237247	total: 1.7s	remaining: 1.57s
520:	learn: 0.3236688	total: 1.71s	remaining: 1.57s
521:	learn: 0.3235727	total: 1.71s	remaining: 1.56s
522:	learn: 0.3234597	total: 1.71s	remaining: 1.56s
523:	learn: 0.3234020	total: 1.72s	remaining: 1.56s
524:	learn: 0.3233680	total: 1.72s	remaining: 1.55s
525:	learn: 0.3232471	total: 1.72s	remaining: 1.55s
526:	learn: 0.3231933	total: 1.73s	remaining: 1.55s
527:	learn: 0.3230553	total: 1.73s	remaining: 1.55s
528:	learn: 0.3228832	total: 1.73s	remaining: 1.54s
529:	learn: 0.3228083	total: 1.74s	remaining: 1.54s
530:	learn: 0.3227249	total: 1.74s	remaining: 1.54s
531:	learn: 0.3224850	total: 1.75s	remaining: 1.54s
532:	learn: 0.3224584	total: 1.75s	remaining: 1.53s
533:	learn: 0.3

733:	learn: 0.3013792	total: 2.37s	remaining: 858ms
734:	learn: 0.3012921	total: 2.37s	remaining: 855ms
735:	learn: 0.3011656	total: 2.37s	remaining: 852ms
736:	learn: 0.3010562	total: 2.38s	remaining: 848ms
737:	learn: 0.3009771	total: 2.38s	remaining: 845ms
738:	learn: 0.3008336	total: 2.38s	remaining: 841ms
739:	learn: 0.3005844	total: 2.38s	remaining: 838ms
740:	learn: 0.3005088	total: 2.39s	remaining: 834ms
741:	learn: 0.3004714	total: 2.39s	remaining: 831ms
742:	learn: 0.3002377	total: 2.39s	remaining: 828ms
743:	learn: 0.3001094	total: 2.4s	remaining: 824ms
744:	learn: 0.2999160	total: 2.4s	remaining: 821ms
745:	learn: 0.2998445	total: 2.4s	remaining: 818ms
746:	learn: 0.2997372	total: 2.4s	remaining: 814ms
747:	learn: 0.2996102	total: 2.41s	remaining: 811ms
748:	learn: 0.2993369	total: 2.41s	remaining: 808ms
749:	learn: 0.2992004	total: 2.41s	remaining: 804ms
750:	learn: 0.2990690	total: 2.42s	remaining: 801ms
751:	learn: 0.2989444	total: 2.42s	remaining: 798ms
752:	learn: 0.29

919:	learn: 0.2813744	total: 2.88s	remaining: 250ms
920:	learn: 0.2812724	total: 2.88s	remaining: 247ms
921:	learn: 0.2812197	total: 2.88s	remaining: 244ms
922:	learn: 0.2809990	total: 2.88s	remaining: 241ms
923:	learn: 0.2809049	total: 2.89s	remaining: 238ms
924:	learn: 0.2808249	total: 2.89s	remaining: 234ms
925:	learn: 0.2807741	total: 2.89s	remaining: 231ms
926:	learn: 0.2807192	total: 2.9s	remaining: 228ms
927:	learn: 0.2805969	total: 2.9s	remaining: 225ms
928:	learn: 0.2804764	total: 2.9s	remaining: 222ms
929:	learn: 0.2803177	total: 2.91s	remaining: 219ms
930:	learn: 0.2801800	total: 2.91s	remaining: 216ms
931:	learn: 0.2800165	total: 2.91s	remaining: 212ms
932:	learn: 0.2799037	total: 2.92s	remaining: 209ms
933:	learn: 0.2797417	total: 2.92s	remaining: 206ms
934:	learn: 0.2795347	total: 2.92s	remaining: 203ms
935:	learn: 0.2794332	total: 2.92s	remaining: 200ms
936:	learn: 0.2793918	total: 2.93s	remaining: 197ms
937:	learn: 0.2793254	total: 2.93s	remaining: 194ms
938:	learn: 0.2

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.838384,81.758700,2.049738
2,CatBoost,83.052750,80.735308,35.348625
3,Decision Tree,82.716049,80.958202,0.315804
4,Skl HistGBM,82.267116,80.649027,13.298621
5,Random Forest,81.930415,80.541175,3.042125
6,LightGBM,81.818182,80.174480,2.024752
7,AdaBoost,81.257015,80.160100,1.645988
8,Extra Trees,81.144781,79.738280,2.884219
9,XGBoost,81.144781,79.628032,2.855241


In [30]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [32]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data

# YOUR CODE HERE
best_model = tree_classifiers['Skl GBM']
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('enc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [33]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [34]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [35]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [36]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"